In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GroupKFold,train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import json
from tqdm import tqdm
# import gc
# import time
pd.set_option('display.max_columns', None)
import os
import os.path as osp
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import display
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload 
%autoreload 2
pd.set_option('display.max_columns', None)
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
df1 = pd.read_csv("Sichuan/train_user.csv")
df2 = pd.read_csv("Sichuan/train_voc.csv")
df3 = pd.read_csv("Sichuan/train_sms.csv")
df4 = pd.read_csv("Sichuan/train_app.csv")
print(df1.shape)
df1.head()
print(df2.shape)
df2.head()
print(df3.shape)
df3.head()
print(df4.shape)
df4.head()

(6106, 13)


,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,46.06,45.00,45.0,45.00,45.00,45.0,45.0,45.0,0
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,79.00,79.20,79.1,79.30,41.40,34.1,59.4,60.0,0
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,18.85,24.45,22.0,21.40,22.15,21.1,61.3,63.0,0
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,19.20,29.65,130.4,173.12,19.00,19.0,19.0,23.2,0
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,50.00,50.00,50.0,50.00,50.00,50.0,50.0,50.0,0


(5015430, 8)


,phone_no_m,opposite_no_m,calltype_id,start_datetime,call_dur,city_name,county_name,imei_m
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,2019-10-17 17:16:25,13,NaN,NaN,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,2019-10-17 17:36:20,16,NaN,NaN,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,c386d11d5203136e924e070afcac734cdaea6691472e84...,1,2019-08-28 09:46:50,18,NaN,NaN,071e8dcb4010c643c48339cb8abf4a69e03b37afade98d...
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,79ef199a532bcb9a89990d0996cee3d40557f06b9e5265...,1,2019-09-18 16:19:20,34,NaN,NaN,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,e273542129cf7fa37c28d3f4c95b7b9e8bcb561bbabc50...,2,2019-09-18 16:18:46,23,NaN,NaN,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...


(6848509, 4)


,phone_no_m,opposite_no_m,calltype_id,request_datetime
0,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
1,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
2,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,1205bb229d750a6bcb3f9c33893b5d68c8fc8a6443b0a7...,2,2019-08-01 17:46:44
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,972affd4be9e7596420379b7e7910843759cdfbd81315e...,2,2019-08-01 16:46:30


(3283602, 4)


,phone_no_m,busi_name,flow,month_id
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,NaN,0.000000,NaN
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,NaN,0.000000,NaN
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,NaN,0.021529,2019-12
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.001846,2019-12
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,高德导航,0.002288,2019-12


In [42]:
# 处理缺失值
for col in df1.columns:
    if np.sum(df1[col].isna()) > 0:
        print(col, np.sum(df1[col].isna()), df1[col].mode().values[0])
        df1[col] = df1[col].fillna(df1[col].mode().values[0])
        print(col, np.sum(df1[col].isna()))
for col in df2.columns:
    if np.sum(df2[col].isna()) > 0:
        print(col, np.sum(df2[col].isna()), df2[col].mode().values[0])
        df2[col] = df2[col].fillna(df2[col].mode().values[0])
        print(col, np.sum(df2[col].isna()))
for col in df3.columns:
    if np.sum(df3[col].isna()) > 0:
        print(col, np.sum(df3[col].isna()), df3[col].mode().values[0])
        df3[col] = df3[col].fillna(df3[col].mode().values[0])
        print(col, np.sum(df3[col].isna()))
for col in df4.columns:
    if np.sum(df4[col].isna()) > 0:
        print(col, np.sum(df4[col].isna()), df4[col].mode().values[0])
        df4[col] = df4[col].fillna(df4[col].mode().values[0])
        print(col, np.sum(df4[col].isna()))


city_name 289 成都
city_name 0
county_name 289 武侯分公司
county_name 0
arpu_201908 1791 1.0
arpu_201908 0
arpu_201909 1641 1.0
arpu_201909 0
arpu_201910 1436 1.0
arpu_201910 0
arpu_201911 1026 1.0
arpu_201911 0
arpu_201912 854 1.0
arpu_201912 0
arpu_202001 957 19.0
arpu_202001 0
arpu_202002 945 19.0
arpu_202002 0
arpu_202003 737 1.0
arpu_202003 0
city_name 559529 成都
city_name 0
county_name 559529 高新南区
county_name 0
busi_name 381835 微信
busi_name 0
month_id 812 2020-01
month_id 0


In [43]:
#不平衡比例
print('诈骗占比：{:.4f}'.format(df1['label'].mean()))

诈骗占比：0.3213


## 特征工程

In [44]:
# 处理时间特征
df2['start_datetime'] = pd.to_datetime(df2['start_datetime'], errors='coerce', infer_datetime_format=True)
df3['request_datetime'] = pd.to_datetime(df3['request_datetime'], errors='coerce', infer_datetime_format=True)
df4['month_id'] = pd.to_datetime(df4['month_id'].astype(str), format='%Y/%m/%d', errors='coerce')

df2['start_year'] = df2['start_datetime'].dt.year
df2['start_month'] = df2['start_datetime'].dt.month
df2['start_day'] = df2['start_datetime'].dt.day
df2['start_hour'] = df2['start_datetime'].dt.hour
df2['start_minute'] = df2['start_datetime'].dt.minute
df2['start_second'] = df2['start_datetime'].dt.second
df2 = df2.drop(columns=['start_datetime'])

df3['request_year'] = df3['request_datetime'].dt.year
df3['request_month'] = df3['request_datetime'].dt.month
df3['request_day'] = df3['request_datetime'].dt.day
df3['request_hour'] = df3['request_datetime'].dt.hour
df3['request_minute'] = df3['request_datetime'].dt.minute
df3['request_second'] = df3['request_datetime'].dt.second
df3 = df3.drop(columns=['request_datetime'])

df4['id_year'] = df4['month_id'].dt.year
df4['id_month'] = df4['month_id'].dt.month
df4['id_day'] = df4['month_id'].dt.day
df4 = df4.drop(columns=['month_id'])


In [45]:
df1.head(5)
df2.head(5)
df3.head(5)
df4.head(5)

,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,46.06,45.00,45.0,45.00,45.00,45.0,45.0,45.0,0
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,79.00,79.20,79.1,79.30,41.40,34.1,59.4,60.0,0
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,18.85,24.45,22.0,21.40,22.15,21.1,61.3,63.0,0
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,19.20,29.65,130.4,173.12,19.00,19.0,19.0,23.2,0
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,50.00,50.00,50.0,50.00,50.00,50.0,50.0,50.0,0


,phone_no_m,opposite_no_m,calltype_id,call_dur,city_name,county_name,imei_m,start_year,start_month,start_day,start_hour,start_minute,start_second
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,13,成都,高新南区,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...,2019,10,17,17,16,25
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,16,成都,高新南区,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...,2019,10,17,17,36,20
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,c386d11d5203136e924e070afcac734cdaea6691472e84...,1,18,成都,高新南区,071e8dcb4010c643c48339cb8abf4a69e03b37afade98d...,2019,8,28,9,46,50
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,79ef199a532bcb9a89990d0996cee3d40557f06b9e5265...,1,34,成都,高新南区,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...,2019,9,18,16,19,20
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,e273542129cf7fa37c28d3f4c95b7b9e8bcb561bbabc50...,2,23,成都,高新南区,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...,2019,9,18,16,18,46


,phone_no_m,opposite_no_m,calltype_id,request_year,request_month,request_day,request_hour,request_minute,request_second
0,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019,8,1,12,13,8
1,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019,8,1,12,13,8
2,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019,8,1,12,13,8
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,1205bb229d750a6bcb3f9c33893b5d68c8fc8a6443b0a7...,2,2019,8,1,17,46,44
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,972affd4be9e7596420379b7e7910843759cdfbd81315e...,2,2019,8,1,16,46,30


,phone_no_m,busi_name,flow,id_year,id_month,id_day
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,微信,0.000000,2020,1,1
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,微信,0.000000,2020,1,1
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.021529,2019,12,1
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.001846,2019,12,1
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,高德导航,0.002288,2019,12,1


In [46]:
# 处理重名变量
df2['city_name_call'] = df2['city_name']
df2['county_name_call'] = df2['county_name']
df2 = df2.drop(columns=['city_name'])
df2 = df2.drop(columns=['county_name'])
df3['sms_id'] = df3['calltype_id']
df3 = df3.drop(columns=['calltype_id'])

categorical_features1 = ['city_name', 'county_name']
categorical_features2 = ['calltype_id', 'city_name_call', 'county_name_call']
categorical_features3 = ['sms_id']
categorical_features4 = ['busi_name']
df1[categorical_features1].nunique()
df2[categorical_features2].nunique()
df3[categorical_features3].nunique()
df4[categorical_features4].nunique()

city_name       22
county_name    181
dtype: int64

calltype_id           3
city_name_call       21
county_name_call    196
dtype: int64

sms_id    2
dtype: int64

busi_name    4994
dtype: int64

In [47]:
# 加入统计信息
sample_df1 = df1
sample_df2 = df2
sample_df3 = df3
sample_df4 = df4

# 添加label
sample_df_list = [sample_df2, sample_df3, sample_df4]
label_mapping = dict(zip(sample_df1['phone_no_m'], sample_df1['label']))
for i, sample_df in enumerate(sample_df_list):
    sample_df['label'] = sample_df['phone_no_m'].map(label_mapping)
    sample_df_list[i] = sample_df

    
cat_features = [categorical_features1, categorical_features2, categorical_features3, categorical_features4]

sample_df_list = [sample_df1, sample_df2, sample_df3, sample_df4]
final_iv_list = []
for i, sample_df in enumerate(sample_df_list):
    for f in cat_features[i]:
        sample_df[f'{f}_cnt'] = sample_df[f].map(sample_df[f].value_counts())
    sample_df.head()
    features = [f for f in sample_df.columns if f not in ['phone_no_m', 'label']]
    final_iv_list.append(WOE(sample_df[[f for f in features if f not in ['opposite_no_m', 'imei_m']]], sample_df.label))
final_iv_list

,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label,city_name_cnt,county_name_cnt
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,46.06,45.00,45.0,45.00,45.00,45.0,45.0,45.0,0,389,81
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,79.00,79.20,79.1,79.30,41.40,34.1,59.4,60.0,0,195,46
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,18.85,24.45,22.0,21.40,22.15,21.1,61.3,63.0,0,2543,55
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,19.20,29.65,130.4,173.12,19.00,19.0,19.0,23.2,0,2543,150
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,50.00,50.00,50.0,50.00,50.00,50.0,50.0,50.0,0,195,46


,phone_no_m,opposite_no_m,calltype_id,call_dur,imei_m,start_year,start_month,start_day,start_hour,start_minute,start_second,city_name_call,county_name_call,label,calltype_id_cnt,city_name_call_cnt,county_name_call_cnt
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,13,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...,2019,10,17,17,16,25,成都,高新南区,0,2000975,2578118,934075
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,16,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...,2019,10,17,17,36,20,成都,高新南区,0,2000975,2578118,934075
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,c386d11d5203136e924e070afcac734cdaea6691472e84...,1,18,071e8dcb4010c643c48339cb8abf4a69e03b37afade98d...,2019,8,28,9,46,50,成都,高新南区,1,3008933,2578118,934075
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,79ef199a532bcb9a89990d0996cee3d40557f06b9e5265...,1,34,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...,2019,9,18,16,19,20,成都,高新南区,0,3008933,2578118,934075
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,e273542129cf7fa37c28d3f4c95b7b9e8bcb561bbabc50...,2,23,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...,2019,9,18,16,18,46,成都,高新南区,0,2000975,2578118,934075


,phone_no_m,opposite_no_m,request_year,request_month,request_day,request_hour,request_minute,request_second,sms_id,label,sms_id_cnt
0,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2019,8,1,12,13,8,2,1,6223623
1,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2019,8,1,12,13,8,2,1,6223623
2,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2019,8,1,12,13,8,2,1,6223623
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,1205bb229d750a6bcb3f9c33893b5d68c8fc8a6443b0a7...,2019,8,1,17,46,44,2,1,6223623
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,972affd4be9e7596420379b7e7910843759cdfbd81315e...,2019,8,1,16,46,30,2,0,6223623


,phone_no_m,busi_name,flow,id_year,id_month,id_day,label,busi_name_cnt
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,微信,0.000000,2020,1,1,1,419966
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,微信,0.000000,2020,1,1,1,419966
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.021529,2019,12,1,1,419966
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.001846,2019,12,1,1,419966
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,高德导航,0.002288,2019,12,1,1,30288


[            VAR_NAME MIN_VALUE MAX_VALUE  COUNT  EVENT  EVENT_RATE  NONEVENT  \
 0          city_name        乐山        乐山    232     68    0.293103       164   
 1          city_name        内江        内江     95     21    0.221053        74   
 2          city_name        凉山        凉山    151     17    0.112583       134   
 3          city_name        南充        南充    211     58    0.274882       153   
 4          city_name      天府新区      天府新区    543    227    0.418048       316   
 ..               ...       ...       ...    ...    ...         ...       ...   
 240  county_name_cnt        21        41   1029    210    0.204082       819   
 241  county_name_cnt        42        62    986    256    0.259635       730   
 242  county_name_cnt        63       133   1084    324    0.298893       760   
 243  county_name_cnt       146       202    990    441    0.445455       549   
 244  county_name_cnt       204       551    971    545    0.561277       426   
 
      NON_EVENT_RATE  DIST

In [48]:
new_timefeatures = [[], ['start_year', 'start_month', 'start_day', 'start_hour', 'start_minute', 'start_second'], ['request_year', 'request_month', 'request_day', 'request_hour', 'request_minute', 'request_second'], ['id_year', 'id_month', 'id_day']]
for index, sample_df in enumerate(sample_df_list):
    #计算特征WOE
    final_iv = final_iv_list[index]
    for f in tqdm(cat_features[index]):
        sample_df[f'{f}_WOE'] = 0
        tmp = final_iv.loc[final_iv['VAR_NAME']==f, :].reset_index()
        for i in range(tmp.shape[0]):
            v = tmp.loc[i, 'MIN_VALUE']
            sample_df.loc[sample_df[f]==v, f'{f}_WOE'] = tmp.loc[i, 'WOE']
    for f in tqdm(new_timefeatures[index]):
        sample_df[f'{f}_WOE'] = 0
        tmp = final_iv.loc[final_iv['VAR_NAME']==f, :].reset_index()
        for i in range(tmp.shape[0]):
            vmin = tmp.loc[i, 'MIN_VALUE']
            vmax = tmp.loc[i, 'MAX_VALUE']
            sample_df.loc[(sample_df[f]<=vmin)&(sample_df[f]>=vmax), f'{f}_WOE'] = tmp.loc[i, 'WOE']
            
    drop = ['city_name', 'county_name', 'city_name_call', 'county_name_call', 'busi_name']
    for column in drop:
        if column in sample_df:
            sample_df = sample_df.drop(columns = column)
            
    sample_df_list[index] = sample_df
    sample_df_list[index]

100%|██████████████████████████| 2/2 [00:00<00:00, 26.62it/s]
0it [00:00, ?it/s]


,phone_no_m,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label,city_name_cnt,county_name_cnt,city_name_WOE,county_name_WOE
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,1,46.06,45.00,45.00,45.00,45.00,45.00,45.00,45.00,0,389,81,-0.410063,0.474404
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,1,79.00,79.20,79.10,79.30,41.40,34.10,59.40,60.00,0,195,46,0.342232,0.213615
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,2,18.85,24.45,22.00,21.40,22.15,21.10,61.30,63.00,0,2543,55,0.440949,-1.352364
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,2,19.20,29.65,130.40,173.12,19.00,19.00,19.00,23.20,0,2543,150,0.440949,0.934909
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,1,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,0,195,46,0.342232,0.213615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6101,bfc115bb229849f256e52e45cac5c400125cfe24c443f1...,1,1.90,2.50,1.80,2.40,1.80,1.60,1.30,1.40,0,188,18,-0.498240,-1.331744
6102,081f09756ed35d349558170213175ceb91b8d1a0392d1d...,4,1.00,1.00,1.00,5.24,1453.50,36.06,22.00,22.00,1,543,160,0.416905,1.049978
6103,fbf1d2b9437078cc8b573528034557f7165cc8ce8776de...,4,1.00,1.00,1.00,1.00,1.00,19.00,19.00,83.14,1,2543,551,0.440949,1.462703
6104,aee363f2f1ff659167a2566d7b466fefd27c907a469590...,3,269.56,243.01,272.60,246.68,259.30,243.29,243.00,257.71,0,389,63,-0.410063,-1.703308


100%|██████████████████████████| 6/6 [00:00<00:00, 15.82it/s]


,phone_no_m,opposite_no_m,calltype_id,call_dur,imei_m,start_year,start_month,start_day,start_hour,start_minute,start_second,label,calltype_id_cnt,city_name_call_cnt,county_name_call_cnt,calltype_id_WOE,city_name_call_WOE,county_name_call_WOE,start_year_WOE,start_month_WOE,start_day_WOE,start_hour_WOE,start_minute_WOE,start_second_WOE
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,13,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...,2019,10,17,17,16,25,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,16,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...,2019,10,17,17,36,20,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,c386d11d5203136e924e070afcac734cdaea6691472e84...,1,18,071e8dcb4010c643c48339cb8abf4a69e03b37afade98d...,2019,8,28,9,46,50,1,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,79ef199a532bcb9a89990d0996cee3d40557f06b9e5265...,1,34,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...,2019,9,18,16,19,20,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,e273542129cf7fa37c28d3f4c95b7b9e8bcb561bbabc50...,2,23,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...,2019,9,18,16,18,46,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5015425,279b5823890d1af8035dadb33f6e8471fdc2832e907d1a...,5825f76d4fb2e58e2ad70af2230ff63d714c00e444bfc0...,1,11,69c38c6bfc7c671103f02560e74eaca480c17900209665...,2019,8,4,20,7,43,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
5015426,279b5823890d1af8035dadb33f6e8471fdc2832e907d1a...,5825f76d4fb2e58e2ad70af2230ff63d714c00e444bfc0...,1,11,69c38c6bfc7c671103f02560e74eaca480c17900209665...,2019,8,4,20,9,54,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
5015427,8a36bc097de313b3f8d9e7843bd89be791baaa8b5ae7f6...,7849800fe2f25372f81ae6a9823c3d1c0c45c6a55c1af6...,2,244,337f125122c651ec9632300620c62f87752dcb7cc6b974...,2019,10,7,18,4,5,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0
5015428,8a36bc097de313b3f8d9e7843bd89be791baaa8b5ae7f6...,e1d2b339f9fb61c517a1cb7fa4033339d6f26017004d0b...,2,61,337f125122c651ec9632300620c62f87752dcb7cc6b974...,2019,10,4,10,59,29,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0.0,0.0,0.0,0.0


100%|██████████████████████████| 6/6 [00:00<00:00, 16.67it/s]


,phone_no_m,opposite_no_m,request_year,request_month,request_day,request_hour,request_minute,request_second,sms_id,label,sms_id_cnt,sms_id_WOE,request_year_WOE,request_month_WOE,request_day_WOE,request_hour_WOE,request_minute_WOE,request_second_WOE
0,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0.0,0.0,0.0,0.0
1,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0.0,0.0,0.0,0.0
2,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0.0,0.0,0.0,0.0
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,1205bb229d750a6bcb3f9c33893b5d68c8fc8a6443b0a7...,2019,8,1,17,46,44,2,1,6223623,-0.291752,0.134422,0.0,0.0,0.0,0.0,0.0
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,972affd4be9e7596420379b7e7910843759cdfbd81315e...,2019,8,1,16,46,30,2,0,6223623,-0.291752,0.134422,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6848504,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2020,3,28,14,29,36,2,0,6223623,-0.291752,-0.220139,0.0,0.0,0.0,0.0,0.0
6848505,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2020,3,28,14,29,36,2,0,6223623,-0.291752,-0.220139,0.0,0.0,0.0,0.0,0.0
6848506,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,f320db4328a1ce096928f9298dac0c26399ca688dd25ff...,2020,3,28,20,12,13,2,0,6223623,-0.291752,-0.220139,0.0,0.0,0.0,0.0,0.0
6848507,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,c8b3458e6e9b632c0c5e32aac0ec9d2bfdec13414ffb2a...,2020,3,28,10,56,47,2,0,6223623,-0.291752,-0.220139,0.0,0.0,0.0,0.0,0.0


100%|██████████████████████████| 3/3 [00:00<00:00, 17.57it/s]


,phone_no_m,flow,id_year,id_month,id_day,label,busi_name_cnt,busi_name_WOE,id_year_WOE,id_month_WOE,id_day_WOE
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,0.000000,2020,1,1,1,419966,-0.037189,-0.102921,0.0,0
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,0.000000,2020,1,1,1,419966,-0.037189,-0.102921,0.0,0
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.021529,2019,12,1,1,419966,-0.037189,0.052483,0.0,0
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.001846,2019,12,1,1,419966,-0.037189,0.052483,0.0,0
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.002288,2019,12,1,1,30288,-0.112531,0.052483,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
3283597,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.015110,2020,1,1,0,6104,0.449036,-0.102921,0.0,0
3283598,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.168604,2020,3,1,0,6104,0.449036,-0.102921,0.0,0
3283599,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.000736,2019,12,1,0,419966,-0.037189,0.052483,0.0,0
3283600,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.000145,2019,12,1,0,419966,-0.037189,0.052483,0.0,0


In [49]:
for index, sample_df in enumerate(sample_df_list):
    for f in ['opposite_no_m', 'imei_m', 'call_event']:
        if f in sample_df:
            sample_df[f'{f}_map'] = sample_df[f].map(dict(zip(sample_df[f].unique(), np.arange(sample_df[f].nunique()))))
            sample_df = sample_df.drop(columns = [f'{f}'])
            sample_df_list[index] = sample_df
            
for index, sample_df in enumerate(sample_df_list):          
    # 丢弃单一取值的变量
    one_cols = []
    for f in sample_df.columns:
        if sample_df[f].nunique() == 1:
            one_cols.append(f)
    print(one_cols)
    sample_df = sample_df.drop(columns=one_cols)
    
    sample_df_list[index] = sample_df
    sample_df_list[index]
    
    

[]


,phone_no_m,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label,city_name_cnt,county_name_cnt,city_name_WOE,county_name_WOE
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,1,46.06,45.00,45.00,45.00,45.00,45.00,45.00,45.00,0,389,81,-0.410063,0.474404
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,1,79.00,79.20,79.10,79.30,41.40,34.10,59.40,60.00,0,195,46,0.342232,0.213615
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,2,18.85,24.45,22.00,21.40,22.15,21.10,61.30,63.00,0,2543,55,0.440949,-1.352364
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,2,19.20,29.65,130.40,173.12,19.00,19.00,19.00,23.20,0,2543,150,0.440949,0.934909
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,1,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,0,195,46,0.342232,0.213615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6101,bfc115bb229849f256e52e45cac5c400125cfe24c443f1...,1,1.90,2.50,1.80,2.40,1.80,1.60,1.30,1.40,0,188,18,-0.498240,-1.331744
6102,081f09756ed35d349558170213175ceb91b8d1a0392d1d...,4,1.00,1.00,1.00,5.24,1453.50,36.06,22.00,22.00,1,543,160,0.416905,1.049978
6103,fbf1d2b9437078cc8b573528034557f7165cc8ce8776de...,4,1.00,1.00,1.00,1.00,1.00,19.00,19.00,83.14,1,2543,551,0.440949,1.462703
6104,aee363f2f1ff659167a2566d7b466fefd27c907a469590...,3,269.56,243.01,272.60,246.68,259.30,243.29,243.00,257.71,0,389,63,-0.410063,-1.703308


['start_day_WOE', 'start_hour_WOE', 'start_minute_WOE', 'start_second_WOE']


,phone_no_m,calltype_id,call_dur,start_year,start_month,start_day,start_hour,start_minute,start_second,label,calltype_id_cnt,city_name_call_cnt,county_name_call_cnt,calltype_id_WOE,city_name_call_WOE,county_name_call_WOE,start_year_WOE,start_month_WOE,opposite_no_m_map,imei_m_map
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,2,13,2019,10,17,17,16,25,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0,0
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,2,16,2019,10,17,17,36,20,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0,0
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,1,18,2019,8,28,9,46,50,1,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,1,1
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,1,34,2019,9,18,16,19,20,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,2,2
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,2,23,2019,9,18,16,18,46,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5015425,279b5823890d1af8035dadb33f6e8471fdc2832e907d1a...,1,11,2019,8,4,20,7,43,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,22,3622
5015426,279b5823890d1af8035dadb33f6e8471fdc2832e907d1a...,1,11,2019,8,4,20,9,54,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,22,3622
5015427,8a36bc097de313b3f8d9e7843bd89be791baaa8b5ae7f6...,2,244,2019,10,7,18,4,5,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,515814,11695
5015428,8a36bc097de313b3f8d9e7843bd89be791baaa8b5ae7f6...,2,61,2019,10,4,10,59,29,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,515860,11695


['request_day_WOE', 'request_hour_WOE', 'request_minute_WOE', 'request_second_WOE']


,phone_no_m,request_year,request_month,request_day,request_hour,request_minute,request_second,sms_id,label,sms_id_cnt,sms_id_WOE,request_year_WOE,request_month_WOE,opposite_no_m_map
0,0251387744988114430181c3e680a3733001a26fe686bd...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0
1,0251387744988114430181c3e680a3733001a26fe686bd...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0
2,0251387744988114430181c3e680a3733001a26fe686bd...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,2019,8,1,17,46,44,2,1,6223623,-0.291752,0.134422,0.0,1
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,2019,8,1,16,46,30,2,0,6223623,-0.291752,0.134422,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6848504,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,2020,3,28,14,29,36,2,0,6223623,-0.291752,-0.220139,0.0,0
6848505,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,2020,3,28,14,29,36,2,0,6223623,-0.291752,-0.220139,0.0,0
6848506,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,2020,3,28,20,12,13,2,0,6223623,-0.291752,-0.220139,0.0,156157
6848507,6aa1b566bffe86325fc9d3c54f7f9abb5c81aa67fb57ee...,2020,3,28,10,56,47,2,0,6223623,-0.291752,-0.220139,0.0,16


['id_day', 'id_month_WOE', 'id_day_WOE']


,phone_no_m,flow,id_year,id_month,label,busi_name_cnt,busi_name_WOE,id_year_WOE
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,0.000000,2020,1,1,419966,-0.037189,-0.102921
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,0.000000,2020,1,1,419966,-0.037189,-0.102921
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.021529,2019,12,1,419966,-0.037189,0.052483
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.001846,2019,12,1,419966,-0.037189,0.052483
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.002288,2019,12,1,30288,-0.112531,0.052483
...,...,...,...,...,...,...,...,...
3283597,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.015110,2020,1,0,6104,0.449036,-0.102921
3283598,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.168604,2020,3,0,6104,0.449036,-0.102921
3283599,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.000736,2019,12,0,419966,-0.037189,0.052483
3283600,2bffd2f47cbd77d187eb95f14a8f17b32fd541bba49721...,0.000145,2019,12,0,419966,-0.037189,0.052483


In [50]:
for index, sample_df in enumerate(sample_df_list):
    sample_df.head()

,phone_no_m,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label,city_name_cnt,county_name_cnt,city_name_WOE,county_name_WOE
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,1,46.06,45.00,45.0,45.00,45.00,45.0,45.0,45.0,0,389,81,-0.410063,0.474404
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,1,79.00,79.20,79.1,79.30,41.40,34.1,59.4,60.0,0,195,46,0.342232,0.213615
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,2,18.85,24.45,22.0,21.40,22.15,21.1,61.3,63.0,0,2543,55,0.440949,-1.352364
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,2,19.20,29.65,130.4,173.12,19.00,19.0,19.0,23.2,0,2543,150,0.440949,0.934909
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,1,50.00,50.00,50.0,50.00,50.00,50.0,50.0,50.0,0,195,46,0.342232,0.213615


,phone_no_m,calltype_id,call_dur,start_year,start_month,start_day,start_hour,start_minute,start_second,label,calltype_id_cnt,city_name_call_cnt,county_name_call_cnt,calltype_id_WOE,city_name_call_WOE,county_name_call_WOE,start_year_WOE,start_month_WOE,opposite_no_m_map,imei_m_map
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,2,13,2019,10,17,17,16,25,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0,0
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,2,16,2019,10,17,17,36,20,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,0,0
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,1,18,2019,8,28,9,46,50,1,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,1,1
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,1,34,2019,9,18,16,19,20,0,3008933,2578118,934075,0.000275,0.457195,1.421664,-0.1785,0.0,2,2
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,2,23,2019,9,18,16,18,46,0,2000975,2578118,934075,0.000000,0.457195,1.421664,-0.1785,0.0,3,2


,phone_no_m,request_year,request_month,request_day,request_hour,request_minute,request_second,sms_id,label,sms_id_cnt,sms_id_WOE,request_year_WOE,request_month_WOE,opposite_no_m_map
0,0251387744988114430181c3e680a3733001a26fe686bd...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0
1,0251387744988114430181c3e680a3733001a26fe686bd...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0
2,0251387744988114430181c3e680a3733001a26fe686bd...,2019,8,1,12,13,8,2,1,6223623,-0.291752,0.134422,0.0,0
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,2019,8,1,17,46,44,2,1,6223623,-0.291752,0.134422,0.0,1
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,2019,8,1,16,46,30,2,0,6223623,-0.291752,0.134422,0.0,2


,phone_no_m,flow,id_year,id_month,label,busi_name_cnt,busi_name_WOE,id_year_WOE
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,0.000000,2020,1,1,419966,-0.037189,-0.102921
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,0.000000,2020,1,1,419966,-0.037189,-0.102921
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.021529,2019,12,1,419966,-0.037189,0.052483
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.001846,2019,12,1,419966,-0.037189,0.052483
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,0.002288,2019,12,1,30288,-0.112531,0.052483


In [51]:
# 节省空间
for index, sample_df in enumerate(sample_df_list):
    sample_df = reduce_mem(sample_df, [f for f in sample_df.columns if f not in ['msisdn', 'train', 'is_sa']])
    
    sample_df_list[index] = sample_df

100%|██████████████████████| 15/15 [00:00<00:00, 2345.20it/s]


0.70 Mb, 0.20 Mb (71.65 %)


100%|████████████████████████| 20/20 [00:01<00:00, 11.02it/s]


765.29 Mb, 224.81 Mb (70.62 %)


100%|████████████████████████| 14/14 [00:01<00:00,  9.69it/s]


731.50 Mb, 202.47 Mb (72.32 %)


100%|██████████████████████████| 8/8 [00:00<00:00, 32.76it/s]


200.42 Mb, 75.16 Mb (62.50 %)


## 合并用户信息

In [52]:
df_list = []
for index, sample_df in enumerate(sample_df_list):
    df = sample_df[[f for f in sample_df.columns if f not in ['label']]].groupby(by='phone_no_m').agg(['mean', 'max', 'min', variation_coefficient, skewness, kurtosis, variance_std_ratio, ratio_beyond_r_sigma, range_ratio])
    df.columns = ['_'.join(col) for col in df.columns]
    df_list.append(df)
df_list

[                                                    idcard_cnt_mean  \
 phone_no_m                                                            
 00073ceecc0f7220a440580ac5dea410c90d14b66694583...              1.0   
 00086f1d2e2c1227f811c3e17f2e9c37cf9971f47bb933b...              4.0   
 000c00db8809c27e723ba90582bf334b2d3ca9063f53fbc...              1.0   
 0014b698069503ceadb9442605834729064be51cdd70024...              1.0   
 0034030646f2008d36431e00b133db30efc8b0c31757f3e...              1.0   
 ...                                                             ...   
 ffc523b3b043cd14a72f03a879beb6794b1065f353800e4...              1.0   
 ffd3a62b9204a8695140a7f04d2bf821c53b1e798fd9382...              1.0   
 ffdbcf9285b71d560a7c79445700db2c363b066003a73d9...              1.0   
 ffdf60d18dbbc25bb8c1697f7c97e8547517b2523d7001a...              1.0   
 ffe9b31003bfabebaad23e4e9eb593cf8566a906c3e4c01...              2.0   
 
                                                     idcard_cn

In [53]:
final_df = df_list[0].copy()

for i, df in enumerate(df_list[1:], start=2):
    columns_to_rename = [col for col in df.columns if col not in ['phone_no_m', 'label']]
    df.rename(columns={col: f"{col}_df{i}" for col in columns_to_rename}, inplace=True)
    
    final_df = pd.merge(final_df, df, on='phone_no_m', how='outer')

if 'label' in final_df.columns:
    final_df = final_df.loc[:, ~final_df.columns.duplicated()]


In [54]:
final_df

,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,idcard_cnt_variation_coefficient,idcard_cnt_skewness,idcard_cnt_kurtosis,idcard_cnt_variance_std_ratio,idcard_cnt_ratio_beyond_r_sigma,idcard_cnt_range_ratio,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201908_variation_coefficient,arpu_201908_skewness,arpu_201908_kurtosis,arpu_201908_variance_std_ratio,arpu_201908_ratio_beyond_r_sigma,arpu_201908_range_ratio,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201909_variation_coefficient,arpu_201909_skewness,arpu_201909_kurtosis,arpu_201909_variance_std_ratio,arpu_201909_ratio_beyond_r_sigma,arpu_201909_range_ratio,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201910_variation_coefficient,arpu_201910_skewness,arpu_201910_kurtosis,arpu_201910_variance_std_ratio,arpu_201910_ratio_beyond_r_sigma,arpu_201910_range_ratio,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201911_variation_coefficient,arpu_201911_skewness,arpu_201911_kurtosis,arpu_201911_variance_std_ratio,arpu_201911_ratio_beyond_r_sigma,arpu_201911_range_ratio,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_201912_variation_coefficient,arpu_201912_skewness,arpu_201912_kurtosis,arpu_201912_variance_std_ratio,arpu_201912_ratio_beyond_r_sigma,arpu_201912_range_ratio,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202001_variation_coefficient,arpu_202001_skewness,arpu_202001_kurtosis,arpu_202001_variance_std_ratio,arpu_202001_ratio_beyond_r_sigma,arpu_202001_range_ratio,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202002_variation_coefficient,arpu_202002_skewness,arpu_202002_kurtosis,arpu_202002_variance_std_ratio,arpu_202002_ratio_beyond_r_sigma,arpu_202002_range_ratio,arpu_202003_mean,arpu_202003_max,arpu_202003_min,arpu_202003_variation_coefficient,arpu_202003_skewness,arpu_202003_kurtosis,arpu_202003_variance_std_ratio,arpu_202003_ratio_beyond_r_sigma,arpu_202003_range_ratio,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,city_name_cnt_variation_coefficient,city_name_cnt_skewness,city_name_cnt_kurtosis,city_name_cnt_variance_std_ratio,city_name_cnt_ratio_beyond_r_sigma,city_name_cnt_range_ratio,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,county_name_cnt_variation_coefficient,county_name_cnt_skewness,county_name_cnt_kurtosis,county_name_cnt_variance_std_ratio,county_name_cnt_ratio_beyond_r_sigma,county_name_cnt_range_ratio,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,city_name_WOE_variation_coefficient,city_name_WOE_skewness,city_name_WOE_kurtosis,city_name_WOE_variance_std_ratio,city_name_WOE_ratio_beyond_r_sigma,city_name_WOE_range_ratio,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,county_name_WOE_variation_coefficient,county_name_WOE_skewness,county_name_WOE_kurtosis,county_name_WOE_variance_std_ratio,county_name_WOE_ratio_beyond_r_sigma,county_name_WOE_range_ratio,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_variance_std_ratio_df2,calltype_id_ratio_beyond_r_sigma_df2,calltype_id_range_ratio_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_variance_std_ratio_df2,start_year_ratio_beyond_r_sigma_df2,start_year_range_ratio_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_variance_std_ratio_df2,start_month_ratio_beyond_r_sigma_df2,start_month_range_ratio_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,s

In [55]:
# 丢弃单一取值的变量
df = final_df
one_cols = []
for f in df.columns:
    if df[f].nunique() == 1:
        one_cols.append(f)
print(one_cols)
df = df.drop(columns=one_cols)

['idcard_cnt_variation_coefficient', 'idcard_cnt_ratio_beyond_r_sigma', 'arpu_201908_variation_coefficient', 'arpu_201908_ratio_beyond_r_sigma', 'arpu_201909_variation_coefficient', 'arpu_201909_ratio_beyond_r_sigma', 'arpu_201910_variation_coefficient', 'arpu_201910_ratio_beyond_r_sigma', 'arpu_201911_variation_coefficient', 'arpu_201911_ratio_beyond_r_sigma', 'arpu_201912_variation_coefficient', 'arpu_201912_ratio_beyond_r_sigma', 'arpu_202001_variation_coefficient', 'arpu_202001_ratio_beyond_r_sigma', 'arpu_202002_variation_coefficient', 'arpu_202002_ratio_beyond_r_sigma', 'arpu_202003_variation_coefficient', 'arpu_202003_ratio_beyond_r_sigma', 'city_name_cnt_variation_coefficient', 'city_name_cnt_ratio_beyond_r_sigma', 'county_name_cnt_variation_coefficient', 'county_name_cnt_ratio_beyond_r_sigma', 'city_name_WOE_variation_coefficient', 'city_name_WOE_ratio_beyond_r_sigma', 'county_name_WOE_variation_coefficient', 'county_name_WOE_ratio_beyond_r_sigma']


In [56]:
df

,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,idcard_cnt_skewness,idcard_cnt_kurtosis,idcard_cnt_variance_std_ratio,idcard_cnt_range_ratio,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201908_skewness,arpu_201908_kurtosis,arpu_201908_variance_std_ratio,arpu_201908_range_ratio,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201909_skewness,arpu_201909_kurtosis,arpu_201909_variance_std_ratio,arpu_201909_range_ratio,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201910_skewness,arpu_201910_kurtosis,arpu_201910_variance_std_ratio,arpu_201910_range_ratio,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201911_skewness,arpu_201911_kurtosis,arpu_201911_variance_std_ratio,arpu_201911_range_ratio,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_201912_skewness,arpu_201912_kurtosis,arpu_201912_variance_std_ratio,arpu_201912_range_ratio,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202001_skewness,arpu_202001_kurtosis,arpu_202001_variance_std_ratio,arpu_202001_range_ratio,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202002_skewness,arpu_202002_kurtosis,arpu_202002_variance_std_ratio,arpu_202002_range_ratio,arpu_202003_mean,arpu_202003_max,arpu_202003_min,arpu_202003_skewness,arpu_202003_kurtosis,arpu_202003_variance_std_ratio,arpu_202003_range_ratio,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,city_name_cnt_skewness,city_name_cnt_kurtosis,city_name_cnt_variance_std_ratio,city_name_cnt_range_ratio,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,county_name_cnt_skewness,county_name_cnt_kurtosis,county_name_cnt_variance_std_ratio,county_name_cnt_range_ratio,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,city_name_WOE_skewness,city_name_WOE_kurtosis,city_name_WOE_variance_std_ratio,city_name_WOE_range_ratio,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,county_name_WOE_skewness,county_name_WOE_kurtosis,county_name_WOE_variance_std_ratio,county_name_WOE_range_ratio,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_variance_std_ratio_df2,calltype_id_ratio_beyond_r_sigma_df2,calltype_id_range_ratio_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_variance_std_ratio_df2,start_year_ratio_beyond_r_sigma_df2,start_year_range_ratio_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_variance_std_ratio_df2,start_month_ratio_beyond_r_sigma_df2,start_month_range_ratio_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,start_day_ratio_beyond_r_sigma_df2,start_day_range_ratio_df2,start_hour_mean_df2,start_hour_max_df2,start_hour_min_df2,start_hour_variation_coefficient_df2,start_hour_skewness_df2,start_hour_kurtosis_df2,start_hour_variance_std_ratio_df2,start_hour_ratio_beyond_r_sigma_df2,start_hour_range_ratio_df2,start_minute_mean_df2,start_minute_max_df2,start_minute_min_df2,start_minute_variation_coefficient_df2,start_minute_skewness_df2,start_minute_kurtosis_df2,start_minute_variance_std_ratio_df2,start_minute_ratio_beyond_r_sigma_df2,start_minute_range_ratio_df2,start_second_mean_df2,start_second_max_df2,start_second_min_df2,start_second_variation_coefficient_df2,start_second_skewness_df2,start_second_kurtosis_df2,start_second_variance_std_ratio_df2,start_second_ratio_beyond_r_sigma_df2,start_second_range_ratio_df2,calltype_id_cnt_mean_df2,calltype_id_cnt_max_df2,calltype_id_cnt_min_df2,call

In [57]:
# 设置阈值 n（百分比）
n = 10  # 指定 NaN 占比的阈值，例如 50%

# 丢弃 NaN 占比大于 n% 的列
nan_cols = [col for col in df.columns if df[col].isnull().mean() * 100 > n]
print(f"NaN 占比大于 {n}% 的列: {nan_cols}")
df = df.drop(columns=nan_cols)

# 对剩余含有 NaN 的列进行填充
for col in df.columns:
    if df[col].isnull().any():
        if df[col].dtype == 'float64' or df[col].dtype == 'int64':
            # 对数值列，使用均值填充
            df[col] = df[col].fillna(df[col].mean())
        else:
            # 对非数值列，使用众数填充
            df[col] = df[col].fillna(df[col].mode()[0])

df

NaN 占比大于 10% 的列: ['idcard_cnt_skewness', 'idcard_cnt_kurtosis', 'idcard_cnt_variance_std_ratio', 'idcard_cnt_range_ratio', 'arpu_201908_skewness', 'arpu_201908_kurtosis', 'arpu_201908_variance_std_ratio', 'arpu_201908_range_ratio', 'arpu_201909_skewness', 'arpu_201909_kurtosis', 'arpu_201909_variance_std_ratio', 'arpu_201909_range_ratio', 'arpu_201910_skewness', 'arpu_201910_kurtosis', 'arpu_201910_variance_std_ratio', 'arpu_201910_range_ratio', 'arpu_201911_skewness', 'arpu_201911_kurtosis', 'arpu_201911_variance_std_ratio', 'arpu_201911_range_ratio', 'arpu_201912_skewness', 'arpu_201912_kurtosis', 'arpu_201912_variance_std_ratio', 'arpu_201912_range_ratio', 'arpu_202001_skewness', 'arpu_202001_kurtosis', 'arpu_202001_variance_std_ratio', 'arpu_202001_range_ratio', 'arpu_202002_skewness', 'arpu_202002_kurtosis', 'arpu_202002_variance_std_ratio', 'arpu_202002_range_ratio', 'arpu_202003_skewness', 'arpu_202003_kurtosis', 'arpu_202003_variance_std_ratio', 'arpu_202003_range_ratio', 'city

,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202003_mean,arpu_202003_max,arpu_202003_min,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_ratio_beyond_r_sigma_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_ratio_beyond_r_sigma_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_ratio_beyond_r_sigma_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,start_day_ratio_beyond_r_sigma_df2,start_day_range_ratio_df2,start_hour_mean_df2,start_hour_max_df2,start_hour_min_df2,start_hour_variation_coefficient_df2,start_hour_skewness_df2,start_hour_kurtosis_df2,start_hour_variance_std_ratio_df2,start_hour_ratio_beyond_r_sigma_df2,start_hour_range_ratio_df2,start_minute_mean_df2,start_minute_max_df2,start_minute_min_df2,start_minute_variation_coefficient_df2,start_minute_skewness_df2,start_minute_kurtosis_df2,start_minute_variance_std_ratio_df2,start_minute_ratio_beyond_r_sigma_df2,start_minute_range_ratio_df2,start_second_mean_df2,start_second_max_df2,start_second_min_df2,start_second_variation_coefficient_df2,start_second_skewness_df2,start_second_kurtosis_df2,start_second_variance_std_ratio_df2,start_second_ratio_beyond_r_sigma_df2,start_second_range_ratio_df2,calltype_id_cnt_mean_df2,calltype_id_cnt_max_df2,calltype_id_cnt_min_df2,calltype_id_cnt_variation_coefficient_df2,calltype_id_cnt_skewness_df2,calltype_id_cnt_kurtosis_df2,calltype_id_cnt_ratio_beyond_r_sigma_df2,city_name_call_cnt_mean_df2,city_name_call_cnt_max_df2,city_name_call_cnt_min_df2,city_name_call_cnt_variation_coefficient_df2,city_name_call_cnt_skewness_df2,city_name_call_cnt_kurtosis_df2,city_name_call_cnt_ratio_beyond_r_sigma_df2,county_name_call_cnt_mean_df2,county_name_call_cnt_max_df2,county_name_call_cnt_min_df2,county_name_call_cnt_variation_coefficient_df2,county_name_call_cnt_skewness_df2,county_name_call_cnt_kurtosis_df2,county_name_call_cnt_ratio_beyond_r_sigma_df2,calltype_id_WOE_mean_df2,calltype_id_WOE_max_df2,calltype_id_WOE_min_df2,calltype_id_WOE_variation_coefficient_df2,calltype_id_WOE_skewness_df2,calltype_id_WOE_kurtosis_df2,calltype_id_WOE_ratio_beyond_r_sigma_df2,city_name_call_WOE_mean_df2,city_name_call_WOE_max_df2,city_name_call_WOE_min_df2,city_name_call_WOE_variation_coefficient_df2,city_name_call_WOE_skewness_df2,city_name_call_WOE_kurtosis_df2,city_name_call_WOE_ratio_beyond_r_sigma_df2,county_name_call_WOE_mean_df2,county_name_call_WOE_max_df2,county_name_call_WOE_min_df2,county_name_call_WOE_variation_coefficient_df2,county_name_call_WOE_skewness_df2,county_name_call_WOE_kurtosis_df2,county_name_call_WOE_ratio_beyond_r_sigma_df2,start_year_WOE_mean_df2,start_year_WOE_max_df2,start_year_WOE_min_df2,start_year_WOE_variation_coefficient_df2,start_year_WOE_skewness_df2,start_year_WOE_kurtosis_df2,start_

In [58]:
df = df.merge(df1[['phone_no_m', 'label']], on='phone_no_m', how='left')
df

,phone_no_m,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202003_mean,arpu_202003_max,arpu_202003_min,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_ratio_beyond_r_sigma_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_ratio_beyond_r_sigma_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_ratio_beyond_r_sigma_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,start_day_ratio_beyond_r_sigma_df2,start_day_range_ratio_df2,start_hour_mean_df2,start_hour_max_df2,start_hour_min_df2,start_hour_variation_coefficient_df2,start_hour_skewness_df2,start_hour_kurtosis_df2,start_hour_variance_std_ratio_df2,start_hour_ratio_beyond_r_sigma_df2,start_hour_range_ratio_df2,start_minute_mean_df2,start_minute_max_df2,start_minute_min_df2,start_minute_variation_coefficient_df2,start_minute_skewness_df2,start_minute_kurtosis_df2,start_minute_variance_std_ratio_df2,start_minute_ratio_beyond_r_sigma_df2,start_minute_range_ratio_df2,start_second_mean_df2,start_second_max_df2,start_second_min_df2,start_second_variation_coefficient_df2,start_second_skewness_df2,start_second_kurtosis_df2,start_second_variance_std_ratio_df2,start_second_ratio_beyond_r_sigma_df2,start_second_range_ratio_df2,calltype_id_cnt_mean_df2,calltype_id_cnt_max_df2,calltype_id_cnt_min_df2,calltype_id_cnt_variation_coefficient_df2,calltype_id_cnt_skewness_df2,calltype_id_cnt_kurtosis_df2,calltype_id_cnt_ratio_beyond_r_sigma_df2,city_name_call_cnt_mean_df2,city_name_call_cnt_max_df2,city_name_call_cnt_min_df2,city_name_call_cnt_variation_coefficient_df2,city_name_call_cnt_skewness_df2,city_name_call_cnt_kurtosis_df2,city_name_call_cnt_ratio_beyond_r_sigma_df2,county_name_call_cnt_mean_df2,county_name_call_cnt_max_df2,county_name_call_cnt_min_df2,county_name_call_cnt_variation_coefficient_df2,county_name_call_cnt_skewness_df2,county_name_call_cnt_kurtosis_df2,county_name_call_cnt_ratio_beyond_r_sigma_df2,calltype_id_WOE_mean_df2,calltype_id_WOE_max_df2,calltype_id_WOE_min_df2,calltype_id_WOE_variation_coefficient_df2,calltype_id_WOE_skewness_df2,calltype_id_WOE_kurtosis_df2,calltype_id_WOE_ratio_beyond_r_sigma_df2,city_name_call_WOE_mean_df2,city_name_call_WOE_max_df2,city_name_call_WOE_min_df2,city_name_call_WOE_variation_coefficient_df2,city_name_call_WOE_skewness_df2,city_name_call_WOE_kurtosis_df2,city_name_call_WOE_ratio_beyond_r_sigma_df2,county_name_call_WOE_mean_df2,county_name_call_WOE_max_df2,county_name_call_WOE_min_df2,county_name_call_WOE_variation_coefficient_df2,county_name_call_WOE_skewness_df2,county_name_call_WOE_kurtosis_df2,county_name_call_WOE_ratio_beyond_r_sigma_df2,start_year_WOE_mean_df2,start_year_WOE_max_df2,start_year_WOE_min_df2,start_year_WOE_variation_coefficient_df2,start_year_WOE_skewness_df2,start_year_WOE_kurtosis

In [59]:
df = reduce_mem(df, [f for f in df.columns if f not in ['msisdn', 'train', 'is_sa']])
df

100%|████████████████████| 300/300 [00:00<00:00, 6432.82it/s]

11.34 Mb, 3.81 Mb (66.38 %)


,phone_no_m,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202003_mean,arpu_202003_max,arpu_202003_min,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_ratio_beyond_r_sigma_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_ratio_beyond_r_sigma_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_ratio_beyond_r_sigma_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,start_day_ratio_beyond_r_sigma_df2,start_day_range_ratio_df2,start_hour_mean_df2,start_hour_max_df2,start_hour_min_df2,start_hour_variation_coefficient_df2,start_hour_skewness_df2,start_hour_kurtosis_df2,start_hour_variance_std_ratio_df2,start_hour_ratio_beyond_r_sigma_df2,start_hour_range_ratio_df2,start_minute_mean_df2,start_minute_max_df2,start_minute_min_df2,start_minute_variation_coefficient_df2,start_minute_skewness_df2,start_minute_kurtosis_df2,start_minute_variance_std_ratio_df2,start_minute_ratio_beyond_r_sigma_df2,start_minute_range_ratio_df2,start_second_mean_df2,start_second_max_df2,start_second_min_df2,start_second_variation_coefficient_df2,start_second_skewness_df2,start_second_kurtosis_df2,start_second_variance_std_ratio_df2,start_second_ratio_beyond_r_sigma_df2,start_second_range_ratio_df2,calltype_id_cnt_mean_df2,calltype_id_cnt_max_df2,calltype_id_cnt_min_df2,calltype_id_cnt_variation_coefficient_df2,calltype_id_cnt_skewness_df2,calltype_id_cnt_kurtosis_df2,calltype_id_cnt_ratio_beyond_r_sigma_df2,city_name_call_cnt_mean_df2,city_name_call_cnt_max_df2,city_name_call_cnt_min_df2,city_name_call_cnt_variation_coefficient_df2,city_name_call_cnt_skewness_df2,city_name_call_cnt_kurtosis_df2,city_name_call_cnt_ratio_beyond_r_sigma_df2,county_name_call_cnt_mean_df2,county_name_call_cnt_max_df2,county_name_call_cnt_min_df2,county_name_call_cnt_variation_coefficient_df2,county_name_call_cnt_skewness_df2,county_name_call_cnt_kurtosis_df2,county_name_call_cnt_ratio_beyond_r_sigma_df2,calltype_id_WOE_mean_df2,calltype_id_WOE_max_df2,calltype_id_WOE_min_df2,calltype_id_WOE_variation_coefficient_df2,calltype_id_WOE_skewness_df2,calltype_id_WOE_kurtosis_df2,calltype_id_WOE_ratio_beyond_r_sigma_df2,city_name_call_WOE_mean_df2,city_name_call_WOE_max_df2,city_name_call_WOE_min_df2,city_name_call_WOE_variation_coefficient_df2,city_name_call_WOE_skewness_df2,city_name_call_WOE_kurtosis_df2,city_name_call_WOE_ratio_beyond_r_sigma_df2,county_name_call_WOE_mean_df2,county_name_call_WOE_max_df2,county_name_call_WOE_min_df2,county_name_call_WOE_variation_coefficient_df2,county_name_call_WOE_skewness_df2,county_name_call_WOE_kurtosis_df2,county_name_call_WOE_ratio_beyond_r_sigma_df2,start_year_WOE_mean_df2,start_year_WOE_max_df2,start_year_WOE_min_df2,start_year_WOE_variation_coefficient_df2,start_year_WOE_skewness_df2,start_year_WOE_kurtosis

In [60]:
mapping = {v: i for i, v in enumerate(df['phone_no_m'].unique(), start=1)}
df['phone_no_m'] = df['phone_no_m'].map(mapping)
df

,phone_no_m,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202003_mean,arpu_202003_max,arpu_202003_min,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_ratio_beyond_r_sigma_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_ratio_beyond_r_sigma_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_ratio_beyond_r_sigma_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,start_day_ratio_beyond_r_sigma_df2,start_day_range_ratio_df2,start_hour_mean_df2,start_hour_max_df2,start_hour_min_df2,start_hour_variation_coefficient_df2,start_hour_skewness_df2,start_hour_kurtosis_df2,start_hour_variance_std_ratio_df2,start_hour_ratio_beyond_r_sigma_df2,start_hour_range_ratio_df2,start_minute_mean_df2,start_minute_max_df2,start_minute_min_df2,start_minute_variation_coefficient_df2,start_minute_skewness_df2,start_minute_kurtosis_df2,start_minute_variance_std_ratio_df2,start_minute_ratio_beyond_r_sigma_df2,start_minute_range_ratio_df2,start_second_mean_df2,start_second_max_df2,start_second_min_df2,start_second_variation_coefficient_df2,start_second_skewness_df2,start_second_kurtosis_df2,start_second_variance_std_ratio_df2,start_second_ratio_beyond_r_sigma_df2,start_second_range_ratio_df2,calltype_id_cnt_mean_df2,calltype_id_cnt_max_df2,calltype_id_cnt_min_df2,calltype_id_cnt_variation_coefficient_df2,calltype_id_cnt_skewness_df2,calltype_id_cnt_kurtosis_df2,calltype_id_cnt_ratio_beyond_r_sigma_df2,city_name_call_cnt_mean_df2,city_name_call_cnt_max_df2,city_name_call_cnt_min_df2,city_name_call_cnt_variation_coefficient_df2,city_name_call_cnt_skewness_df2,city_name_call_cnt_kurtosis_df2,city_name_call_cnt_ratio_beyond_r_sigma_df2,county_name_call_cnt_mean_df2,county_name_call_cnt_max_df2,county_name_call_cnt_min_df2,county_name_call_cnt_variation_coefficient_df2,county_name_call_cnt_skewness_df2,county_name_call_cnt_kurtosis_df2,county_name_call_cnt_ratio_beyond_r_sigma_df2,calltype_id_WOE_mean_df2,calltype_id_WOE_max_df2,calltype_id_WOE_min_df2,calltype_id_WOE_variation_coefficient_df2,calltype_id_WOE_skewness_df2,calltype_id_WOE_kurtosis_df2,calltype_id_WOE_ratio_beyond_r_sigma_df2,city_name_call_WOE_mean_df2,city_name_call_WOE_max_df2,city_name_call_WOE_min_df2,city_name_call_WOE_variation_coefficient_df2,city_name_call_WOE_skewness_df2,city_name_call_WOE_kurtosis_df2,city_name_call_WOE_ratio_beyond_r_sigma_df2,county_name_call_WOE_mean_df2,county_name_call_WOE_max_df2,county_name_call_WOE_min_df2,county_name_call_WOE_variation_coefficient_df2,county_name_call_WOE_skewness_df2,county_name_call_WOE_kurtosis_df2,county_name_call_WOE_ratio_beyond_r_sigma_df2,start_year_WOE_mean_df2,start_year_WOE_max_df2,start_year_WOE_min_df2,start_year_WOE_variation_coefficient_df2,start_year_WOE_skewness_df2,start_year_WOE_kurtosis

In [61]:
df = df.replace(np.inf,1e10)
df = df.replace(-np.inf,-1e10)
# 查看df是否有缺失值和无穷大值
for f in df.columns:
    if df[f].isna().sum()>0:
        print('缺失值',f, df[f].isna().sum())
    if np.sum(np.isinf(df[f].values)) > 0:
        print('无穷大值',f, np.sum(np.isinf(df[f].values)))

In [62]:
np.random.seed(42)

# 计算总行数
n_rows = len(df)

# 生成随机分布
train_values = np.random.choice([1, 0], size=n_rows, p=[0.8, 0.2])

# 添加到 DataFrame
df['train'] = train_values
df

,phone_no_m,idcard_cnt_mean,idcard_cnt_max,idcard_cnt_min,arpu_201908_mean,arpu_201908_max,arpu_201908_min,arpu_201909_mean,arpu_201909_max,arpu_201909_min,arpu_201910_mean,arpu_201910_max,arpu_201910_min,arpu_201911_mean,arpu_201911_max,arpu_201911_min,arpu_201912_mean,arpu_201912_max,arpu_201912_min,arpu_202001_mean,arpu_202001_max,arpu_202001_min,arpu_202002_mean,arpu_202002_max,arpu_202002_min,arpu_202003_mean,arpu_202003_max,arpu_202003_min,city_name_cnt_mean,city_name_cnt_max,city_name_cnt_min,county_name_cnt_mean,county_name_cnt_max,county_name_cnt_min,city_name_WOE_mean,city_name_WOE_max,city_name_WOE_min,county_name_WOE_mean,county_name_WOE_max,county_name_WOE_min,calltype_id_mean_df2,calltype_id_max_df2,calltype_id_min_df2,calltype_id_variation_coefficient_df2,calltype_id_skewness_df2,calltype_id_kurtosis_df2,calltype_id_ratio_beyond_r_sigma_df2,call_dur_mean_df2,call_dur_max_df2,call_dur_min_df2,call_dur_variation_coefficient_df2,call_dur_skewness_df2,call_dur_kurtosis_df2,call_dur_variance_std_ratio_df2,call_dur_ratio_beyond_r_sigma_df2,call_dur_range_ratio_df2,start_year_mean_df2,start_year_max_df2,start_year_min_df2,start_year_variation_coefficient_df2,start_year_skewness_df2,start_year_kurtosis_df2,start_year_ratio_beyond_r_sigma_df2,start_month_mean_df2,start_month_max_df2,start_month_min_df2,start_month_variation_coefficient_df2,start_month_skewness_df2,start_month_kurtosis_df2,start_month_ratio_beyond_r_sigma_df2,start_day_mean_df2,start_day_max_df2,start_day_min_df2,start_day_variation_coefficient_df2,start_day_skewness_df2,start_day_kurtosis_df2,start_day_variance_std_ratio_df2,start_day_ratio_beyond_r_sigma_df2,start_day_range_ratio_df2,start_hour_mean_df2,start_hour_max_df2,start_hour_min_df2,start_hour_variation_coefficient_df2,start_hour_skewness_df2,start_hour_kurtosis_df2,start_hour_variance_std_ratio_df2,start_hour_ratio_beyond_r_sigma_df2,start_hour_range_ratio_df2,start_minute_mean_df2,start_minute_max_df2,start_minute_min_df2,start_minute_variation_coefficient_df2,start_minute_skewness_df2,start_minute_kurtosis_df2,start_minute_variance_std_ratio_df2,start_minute_ratio_beyond_r_sigma_df2,start_minute_range_ratio_df2,start_second_mean_df2,start_second_max_df2,start_second_min_df2,start_second_variation_coefficient_df2,start_second_skewness_df2,start_second_kurtosis_df2,start_second_variance_std_ratio_df2,start_second_ratio_beyond_r_sigma_df2,start_second_range_ratio_df2,calltype_id_cnt_mean_df2,calltype_id_cnt_max_df2,calltype_id_cnt_min_df2,calltype_id_cnt_variation_coefficient_df2,calltype_id_cnt_skewness_df2,calltype_id_cnt_kurtosis_df2,calltype_id_cnt_ratio_beyond_r_sigma_df2,city_name_call_cnt_mean_df2,city_name_call_cnt_max_df2,city_name_call_cnt_min_df2,city_name_call_cnt_variation_coefficient_df2,city_name_call_cnt_skewness_df2,city_name_call_cnt_kurtosis_df2,city_name_call_cnt_ratio_beyond_r_sigma_df2,county_name_call_cnt_mean_df2,county_name_call_cnt_max_df2,county_name_call_cnt_min_df2,county_name_call_cnt_variation_coefficient_df2,county_name_call_cnt_skewness_df2,county_name_call_cnt_kurtosis_df2,county_name_call_cnt_ratio_beyond_r_sigma_df2,calltype_id_WOE_mean_df2,calltype_id_WOE_max_df2,calltype_id_WOE_min_df2,calltype_id_WOE_variation_coefficient_df2,calltype_id_WOE_skewness_df2,calltype_id_WOE_kurtosis_df2,calltype_id_WOE_ratio_beyond_r_sigma_df2,city_name_call_WOE_mean_df2,city_name_call_WOE_max_df2,city_name_call_WOE_min_df2,city_name_call_WOE_variation_coefficient_df2,city_name_call_WOE_skewness_df2,city_name_call_WOE_kurtosis_df2,city_name_call_WOE_ratio_beyond_r_sigma_df2,county_name_call_WOE_mean_df2,county_name_call_WOE_max_df2,county_name_call_WOE_min_df2,county_name_call_WOE_variation_coefficient_df2,county_name_call_WOE_skewness_df2,county_name_call_WOE_kurtosis_df2,county_name_call_WOE_ratio_beyond_r_sigma_df2,start_year_WOE_mean_df2,start_year_WOE_max_df2,start_year_WOE_min_df2,start_year_WOE_variation_coefficient_df2,start_year_WOE_skewness_df2,start_year_WOE_kurtosis

In [63]:
from copy import deepcopy
pre_dir='./data/0720'
df.rename(columns={'label': 'is_sa'}, inplace=True)

train = df[df['train'] == 1].reset_index(drop=True)
testb = df[df['train'] == 0].reset_index(drop=True)
cols = [
    f for f in train.columns if f not in [
        'phone_no_m', 'train', 'is_sa'
    ] 
]
train[cols].shape
testb[cols].shape
train_data= train[cols].values
train_y= train['is_sa'].values
test_data= testb[cols].values
test_y= testb['is_sa'].values
test_name=testb['phone_no_m']
# 划分验证集
# train_data,val_data,train_y,val_y=train_test_split(train_data,train_y,test_size=0.2, random_state=42,stratify=train_y)
save_dir='./data/SiChuan'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
np.save(osp.join(save_dir, 'N_train.npy'), train_data)
# np.save(osp.join(save_dir, 'N_val.npy'), val_data)
np.save(osp.join(save_dir, 'N_test.npy'), test_data)
np.save(osp.join(save_dir, 'y_train.npy'), train_y)
np.save(osp.join(save_dir, 'y_test.npy'), train_y)
# np.save(osp.join(save_dir, 'y_val.npy'), val_y)

test_output = pd.DataFrame({
    'phone_no_m': test_name,
    'is_sa': test_y
})
test_output.to_csv(osp.join(save_dir, 'test_true.csv'), index=False)
pd.DataFrame(test_name).to_csv(osp.join(save_dir, 'test_msisdn.csv'), index=False)

#save feature name
feature_name = cols
with open(osp.join(save_dir, 'feature_name.txt'), 'w') as f:
    for i in feature_name:
        f.write(i+'\n')
info={}
info["name"] = "TeleChallenge"
info["n_num_features"] = train_data.shape[1]
info["n_classes"] = 2
info["task_type"] ="binclass"
info["n_cat_features"]=0
info["train_size"]=train_data.shape[0]
# info["val_size"]=val_data.shape[0]
info["test_size"]=test_data.shape[0]
with open(osp.join(save_dir, 'info.json'), 'w') as f:
    json.dump(info, f,indent=4)

(4886, 298)

(1220, 298)

16

15

15

...

21

20

20

38

37